In [0]:
# %run ../00_config

In [0]:
print("渡航前アンケートのCSVファイルを作成します...")

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import to_timestamp, col
import random, datetime

# ============================================================
# テーブル: pre_survey（渡航前アンケート）サンプル
# ============================================================

# ---------- 設定 ----------
N_USERS               = 3000      # 会員総数
APP_USAGE_RATE        = 0.70     # アプリ利用率（70%）
SURVEY_RESPONSE_RATE  = 0.85     # 利用者のうち回答率（85%）
MIN_RESP_PER_USER     = 1        # 同一ユーザーの最小回答回数
MAX_RESP_PER_USER     = 3        # 同一ユーザーの最大回答回数

trip_purpose_choices = ["仕事", "観光"]
content_category_choices = [
    "観光情報",
    "食事・グルメ情報",
    "移送手段や便利な情報",
    "リラックスできるコンテンツ",
    "文化やマナーに関する情報",
    "映画、ドラマ",
]

# ---------- 回答者を決定 ----------
# ① 会員400名のうち 60% がアプリを利用
# ② そのうち 75% がアンケート回答
n_app_users    = int(N_USERS * APP_USAGE_RATE)
n_survey_users = int(n_app_users * SURVEY_RESPONSE_RATE)
respondents    = random.sample(range(1, N_USERS + 1), n_survey_users)

# ---------- 日時生成パラメータ ----------
start_dt  = datetime.datetime(2024, 1, 1, 0, 0, 0)
end_dt    = datetime.datetime(2025, 12, 31, 23, 59, 59)
total_sec = int((end_dt - start_dt).total_seconds())

def rand_answer_dt() -> str:
    '''
    論理名：回答日時
    物理名：answer_date
    ルール：2024-01-01 〜 2025-12-31 のランダム日時
    '''
    return (start_dt + datetime.timedelta(seconds=random.randint(0, total_sec))
            ).strftime("%Y-%m-%d %H:%M:%S")

def rand_trip_purpose() -> str:
    '''
    論理名：旅行目的
    物理名：trip_purpose
    ルール：['仕事', '観光'] からランダム
    '''
    return random.choice(trip_purpose_choices)

def rand_content_category() -> str:
    '''
    論理名：機内エンタメカテゴリ
    物理名：content_category
    ルール：6 つのカテゴリからランダム
    '''
    return random.choice(content_category_choices)

# ---------- 行レコードを組み立て ----------
rows = []
for uid in respondents:
    # ユーザーごとに 1〜3 回回答
    n_resp = random.randint(MIN_RESP_PER_USER, MAX_RESP_PER_USER)
    for _ in range(n_resp):
        rows.append(
            Row(
                user_id          = uid,                  # ★ 会員ID（数値型に変更）
                trip_purpose     = rand_trip_purpose(),    # 旅行目的
                content_category = rand_content_category(),# エンタメカテゴリ
                answer_date      = rand_answer_dt(),       # 回答日時
            )
        )

# ---------- DataFrame 化 ----------
df = spark.createDataFrame(rows) \
    .withColumn(                   # answer_date を TIMESTAMP 型に
        "answer_date",
        to_timestamp(col("answer_date"), "yyyy-MM-dd HH:mm:ss")
    )

# ---------- 動作確認 ----------
print("回答レコード数:", df.count())          # 例: 300〜500 行程度
# df.show(10, truncate=False)
df.printSchema()
print("各ユーザーのアンケート回答数分布（1〜3 回）")
df.groupBy("user_id").count() \
  .groupBy("count").count() \
  .orderBy("count").show()                   # 1〜3 回の分布確認

# ---------- Delta テーブル保存 ----------
# df.write.format("delta")\
#   .option("comment", "機内エンタメレコメンドのための渡航前アンケート")\
#   .mode("overwrite")\
#   .saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.bz_pre_survey")

# ---------- CSV 出力 ----------
out_path = f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/pre_survey/pre_survey.csv"
df.coalesce(1).toPandas().to_csv(out_path, index=False)
print(out_path)

In [0]:
print("渡航前アンケートのCSVファイルを作成しました！")